# Phase 7: Bayesian Robustness Check

**Swiss Ballot Chatbot Study — Supplementary Analysis**

This notebook computes Bayes Factors (BF₁₀) for the three main hypotheses as a
complement to the frequentist tests reported in Phases 2–3. The goal is to
distinguish *absence of evidence* from *evidence of absence* by quantifying
relative support for H₁ vs. H₀.

## Method

For each hypothesis the dependent variable is binary (donate / decline).
We use an analytic **beta-binomial model** (conjugate prior):

| Model | Assumption |
|-------|------------|
| H₀   | Both groups share a single donation rate *p* ~ Beta(1, 1) |
| H₁   | Each group has its own rate *p₁*, *p₂* ~ Beta(1, 1) independently |

The Bayes Factor is the ratio of marginal likelihoods:

$$BF_{10} = \frac{B(s_1+1,\, f_1+1)\; B(s_2+1,\, f_2+1)}{B(s_1+s_2+1,\, f_1+f_2+1)}$$

where *s* = donations, *f* = declines, and *B* is the beta function.

### Interpretation (Jeffreys, 1961)

| BF₁₀ | Evidence |
|-------|----------|
| > 10  | Strong for H₁ |
| 3–10  | Moderate for H₁ |
| 1–3   | Anecdotal for H₁ |
| 1/3–1 | Anecdotal for H₀ |
| 1/10–1/3 | Moderate for H₀ |
| < 1/10 | Strong for H₀ |

---

## Setup

In [ ]:
import numpy as np
import pandas as pd
from scipy.special import betaln
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 100

from phase1_descriptive_statistics import (
    AnalysisConfig,
    load_participant_data,
    prepare_variables,
    compute_sample_flow
)

print('Libraries loaded.')

In [ ]:
# Load data (human participants)
config = AnalysisConfig(is_ai_participant=False)
df_raw = load_participant_data(config)
df = prepare_variables(df_raw, config)
df_filtered = compute_sample_flow(df)['df_filtered']

print(f'Final sample: N = {len(df_filtered)}')

---
## Bayes Factor computation

In [ ]:
def bf10_two_proportions(s1, f1, s2, f2):
    """Bayes Factor for two independent proportions (beta-binomial conjugate model).

    H0: both groups share one rate  p  ~ Beta(1,1)
    H1: each group has its own rate p_i ~ Beta(1,1)

    Parameters
    ----------
    s1, f1 : int  — successes and failures in group 1
    s2, f2 : int  — successes and failures in group 2

    Returns
    -------
    bf10 : float  — P(data | H1) / P(data | H0)
    """
    log_bf10 = (
        betaln(s1 + 1, f1 + 1)
        + betaln(s2 + 1, f2 + 1)
        - betaln(s1 + s2 + 1, f1 + f2 + 1)
        - betaln(1, 1)          # = 0, kept for clarity
    )
    return np.exp(log_bf10)


def interpret_bf(bf10):
    """Jeffreys (1961) interpretation."""
    if bf10 > 100:   return 'Extreme evidence for H\u2081'
    if bf10 > 30:    return 'Very strong evidence for H\u2081'
    if bf10 > 10:    return 'Strong evidence for H\u2081'
    if bf10 > 3:     return 'Moderate evidence for H\u2081'
    if bf10 > 1:     return 'Anecdotal evidence for H\u2081'
    if bf10 > 1/3:   return 'Anecdotal evidence for H\u2080'
    if bf10 > 1/10:  return 'Moderate evidence for H\u2080'
    if bf10 > 1/30:  return 'Strong evidence for H\u2080'
    if bf10 > 1/100: return 'Very strong evidence for H\u2080'
    return 'Extreme evidence for H\u2080'


print('Functions defined.')

---
## H1 — Transparency (T0 vs T1)

In [ ]:
# H1: Transparency
t0 = df_filtered[df_filtered['transparency_level'] == 0]['donation_decision']
t1 = df_filtered[df_filtered['transparency_level'] == 1]['donation_decision']

s1_h1, f1_h1 = int(t0.sum()), int(len(t0) - t0.sum())
s2_h1, f2_h1 = int(t1.sum()), int(len(t1) - t1.sum())

bf10_h1 = bf10_two_proportions(s1_h1, f1_h1, s2_h1, f2_h1)

print('H1 — Transparency Effect')
print(f'  T0 (Low):  {s1_h1} donated, {f1_h1} declined  (n={s1_h1+f1_h1})')
print(f'  T1 (High): {s2_h1} donated, {f2_h1} declined  (n={s2_h1+f2_h1})')
print(f'  BF\u2081\u2080 = {bf10_h1:.4f}')
print(f'  BF\u2080\u2081 = {1/bf10_h1:.2f}')
print(f'  → {interpret_bf(bf10_h1)}')

---
## H2 — Control (C0 vs C1)

In [ ]:
# H2: Control
c0 = df_filtered[df_filtered['control_level'] == 0]['donation_decision']
c1 = df_filtered[df_filtered['control_level'] == 1]['donation_decision']

s1_h2, f1_h2 = int(c0.sum()), int(len(c0) - c0.sum())
s2_h2, f2_h2 = int(c1.sum()), int(len(c1) - c1.sum())

bf10_h2 = bf10_two_proportions(s1_h2, f1_h2, s2_h2, f2_h2)

print('H2 — Control Effect')
print(f'  C0 (Low):  {s1_h2} donated, {f1_h2} declined  (n={s1_h2+f1_h2})')
print(f'  C1 (High): {s2_h2} donated, {f2_h2} declined  (n={s2_h2+f2_h2})')
print(f'  BF\u2081\u2080 = {bf10_h2:.4f}')
print(f'  BF\u2080\u2081 = {1/bf10_h2:.2f}')
print(f'  → {interpret_bf(bf10_h2)}')

---
## H3 — Interaction (T × C)

For the interaction, we compare a model where the four conditions have
independent rates against a model where all share one rate. This yields
a global BF for the 4-condition comparison.

In [ ]:
def bf10_k_proportions(*groups):
    """Bayes Factor for k independent proportions vs. a common proportion.

    H0: all groups share one rate  p  ~ Beta(1,1)
    H1: each group has its own rate p_i ~ Beta(1,1)

    Parameters
    ----------
    *groups : tuple of (successes, failures) for each group

    Returns
    -------
    bf10 : float
    """
    log_h1 = sum(betaln(s + 1, f + 1) for s, f in groups)
    total_s = sum(s for s, _ in groups)
    total_f = sum(f for _, f in groups)
    log_h0 = betaln(total_s + 1, total_f + 1)
    # Normalizing constants: k Beta(1,1) for H1, 1 Beta(1,1) for H0
    k = len(groups)
    log_bf10 = log_h1 - log_h0 - (k - 1) * betaln(1, 1)
    return np.exp(log_bf10)


# Four conditions
groups = {}
for cond in ['A', 'B', 'C', 'D']:
    sub = df_filtered[df_filtered['condition'] == cond]['donation_decision']
    groups[cond] = (int(sub.sum()), int(len(sub) - sub.sum()))

bf10_h3 = bf10_k_proportions(*groups.values())

print('H3 — Interaction (4-condition comparison)')
for cond, (s, f) in groups.items():
    print(f'  {cond}: {s} donated, {f} declined  (n={s+f}, rate={s/(s+f)*100:.1f}%)')
print(f'\n  BF\u2081\u2080 = {bf10_h3:.4f}')
print(f'  BF\u2080\u2081 = {1/bf10_h3:.2f}')
print(f'  → {interpret_bf(bf10_h3)}')

---
## Summary

In [ ]:
# Summary table
summary = pd.DataFrame([
    {
        'Hypothesis': 'H1 (Transparency)',
        'Comparison': 'T0 vs T1',
        'BF\u2081\u2080': round(bf10_h1, 4),
        'BF\u2080\u2081': round(1 / bf10_h1, 2),
        'Interpretation': interpret_bf(bf10_h1)
    },
    {
        'Hypothesis': 'H2 (Control)',
        'Comparison': 'C0 vs C1',
        'BF\u2081\u2080': round(bf10_h2, 4),
        'BF\u2080\u2081': round(1 / bf10_h2, 2),
        'Interpretation': interpret_bf(bf10_h2)
    },
    {
        'Hypothesis': 'H3 (Interaction)',
        'Comparison': 'A vs B vs C vs D',
        'BF\u2081\u2080': round(bf10_h3, 4),
        'BF\u2080\u2081': round(1 / bf10_h3, 2),
        'Interpretation': interpret_bf(bf10_h3)
    }
])

print('=' * 90)
print('BAYESIAN ROBUSTNESS CHECK — SUMMARY')
print('=' * 90)
print(summary.to_string(index=False))
print('\nNote: BF\u2080\u2081 > 3 = moderate evidence for the null hypothesis.')

# Save
summary.to_csv('output/phase7/phase7_bayesian_summary.csv', index=False)
print('\nSaved: output/phase7/phase7_bayesian_summary.csv')

In [ ]:
# Visualization: BF₀₁ (evidence for null) on log scale
fig, ax = plt.subplots(figsize=(8, 4))

hypotheses = ['H1\nTransparency', 'H2\nControl', 'H3\nInteraction']
bf01_values = [1/bf10_h1, 1/bf10_h2, 1/bf10_h3]
colors = ['#e74c3c', '#9b59b6', '#3498db']

bars = ax.barh(hypotheses, bf01_values, color=colors, edgecolor='black', linewidth=1.2)

# Reference lines
ax.axvline(x=3, color='gray', linestyle='--', linewidth=1, label='Moderate (BF\u2080\u2081 = 3)')
ax.axvline(x=10, color='gray', linestyle=':', linewidth=1, label='Strong (BF\u2080\u2081 = 10)')

for bar, val in zip(bars, bf01_values):
    ax.text(bar.get_width() + 0.3, bar.get_y() + bar.get_height()/2,
            f'{val:.1f}', va='center', fontsize=11, fontweight='bold')

ax.set_xlabel('BF\u2080\u2081 (Evidence for H\u2080)', fontsize=12)
ax.set_xlim(0, max(bf01_values) * 1.3)
ax.legend(loc='lower right', fontsize=9)

plt.tight_layout()
plt.savefig('output/phase7/fig_bayesian_bf01.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: output/phase7/fig_bayesian_bf01.png')